In [21]:
from typing import List, Tuple, Optional, Dict, NamedTuple, Union, Callable
from json import loads, dumps
import itertools
import os
import string
from pathlib import Path

import numpy as np
import pandas as pd
import torch

from Bio import SeqIO

from tqdm import tqdm

# https://github.com/facebookresearch/esm
# pip install fair-esm
import esm

from utils.file import save_np, csv2df, read_json2list, dump_list2json
from utils.alignmentParser import read_msa, greedy_select
from params.PDBparser import *

torch.set_grad_enabled(False)

'''
Aim to embed the CAID pdb_disorder using MSA-Transformer. And save them to files.

Di
16 Feb, 2024
'''
# 1. mdoel & tokenizer
msa_transformer, msa_transformer_alphabet = esm.pretrained.esm_msa1b_t12_100M_UR50S()
msa_transformer = msa_transformer.eval()
msa_transformer_batch_converter = msa_transformer_alphabet.get_batch_converter()

# 2. predict & save embedded sequences
path_hmm = '/home/dimeng/project/idr/data/caid/a3m'

df_caid= pd.DataFrame(read_json2list(path_caid_dataset_json)) # entities' features

# df_caid['len_1022'] = [1 if l<=1022 else 0 for l in df_caid['length']]

# entyID_entityID
seq_IDS = ['DP02411']

In [23]:
df_caid = df_caid[(df_caid['len_1022']==0)&(df_caid['length']<=1022)]
df_caid.shape

(56, 7)

In [8]:
df_caid[df_caid['id']=='DP02411']

,id,sequence,length,reference,info_idr,num_idr,len_1022
9,DP02411,MKTSPRRPLILKRRRLPLPVQNAPSETSEEEPKRSPAQQESNQAEA...,763,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{'1': {'start': 0, 'end': 113, 'len_linker': 1...",4,1


In [11]:
name = seq_IDS[0]
length = 763
inputs = read_msa(os.path.join(path_hmm, f'{name}.a3m'))

In [13]:
inputs[0][1] = inputs[0][1][:length]

TypeError: 'tuple' object does not support item assignment

In [16]:
inputs = [(i[0], i[1][:length]) for i in inputs]
df_caid.iterrows()

In [17]:
inputs

[('DP02411',
  'MKTSPRRPLILKRRRLPLPVQNAPSETSEEEPKRSPAQQESNQAEASKEVAESNSCKFPAGIKIINHPTMPNTQVVAIPNNANIHSIITALTAKGKESGSSGPNKFILISCGGAPTQPPGLRPQTQTSYDAKRTEVTLETLGPKPAARDVNLPRPPGALCEQKRETCADGEAAGCTINNSLSNIQWLRKMSSDGLGSRSIKQEMEEKENCHLEQRQVKVEEPSRPSASWQNSVSERPPYSYMAMIQFAINSTERKRMTLKDIYTWIEDHFPYFKHIAKPGWKNSIRHNLSLHDMFVRETSANGKVSFWTIHPSANRYLTLDQVFKPLDPGSPQLPEHLESQQKRPNPELRRNMTIKTELPLGARRKMKPLLPRVSSYLVPIQFPVNQSLVLQPSVKVPLPLAASLMSSELARHSKRVRIAPKVLLAEEGIAPLSSAGPGKEEKLLFGEGFSPLLPVQTIKEEEIQPGEEMPHLARPIKVESPPLEEWPSPAPSFKEESSHSWEDSSQSPTPRPKKSYSGLRSPTRCVSEMLVIQHRERRERSRSRRKQHLLPPCVDEPELLFSEGPSTSRWAAELPFPADSSDPASQLSYSQEVGGPFKTPIKETLPISSTPSKSVLPRTPESWRLTPPAKVGGLDFSPVQTSQGASDPLPDPLGLMDLSTTPLQSAPPLESPQRLLSSEPLDLISVPFGNSSPSDIDVPKPGSPEPQVSGLAANRSLTEGLVLDTMNDSLSKILLDISFPGLDEDPLGPDNINWSQFIPELQ'),
 ('UniRef100_A0A091CR53 Forkhead box protein M1 n=1 Tax=Fukomys damarensis TaxID=885580 RepID=A0A091CR53_FUKDA',
  'MKTSPHRPLILKRRRLPLPVQNAPAETSKEECKKPPAQKEPSQAPASKEVAESSSCKFPAGIKIINHPTMPNTQVVAIPSSADIQSIITVLTAKGKESGSS

In [ ]:
name = seq_IDS[0]
inputs = read_msa(os.path.join(path_hmm, f'{name}.a3m'))
if len(inputs[0][1])>1022:
    print(f'Sequence length ({len(inputs[0][1])}) is greater than 1022. ')
    df_caid.loc[df_caid['id']==name, 'len_1022'] = 0
    continue